In [ ]:
%matplotlib widget
import logging
import warnings

import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import symplot
import sympy as sp
from ampform.dynamics import (
    BlattWeisskopfSquared,
    breakup_momentum_squared,
    coupled_width,
    phase_space_factor,
    phase_space_factor_analytic,
    phase_space_factor_complex,
    relativistic_breit_wigner,
    relativistic_breit_wigner_with_ff,
)
from ampform.dynamics.math import ComplexSqrt
from IPython.display import Math
from ipywidgets import widgets

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

np.seterr(divide="ignore");

In [ ]:
%%HTML
<style>
div.prompt {display:none}
</style>

# Analytic continuation

In [ ]:
m, m0, w0, m_a, m_b, d = sp.symbols("m m0 Gamma0 m_a m_b d")
L = sp.Symbol("L", integer=True)
s = m ** 2

In [ ]:
rel_bw = relativistic_breit_wigner(
    s=s,
    mass0=m0,
    gamma0=w0,
)
rel_bw_with_ff = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor,
)

running_width = coupled_width(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor,
)

Non-relativistic Breit-Wigner:

In [ ]:
rel_bw

Relativistic Breit-Wigner with form factor _above threshold only_ [[PDG2020, §Resonances, p.6](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=6)]:

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
ff_sq = BlattWeisskopfSquared(L, z=q_squared * d ** 2)
rel_bw_with_ff_subs = rel_bw_with_ff.subs(
    {
        2 * q_squared: 2 * sp.Symbol("q^{2}(m)"),
        ff_sq: sp.Symbol(R"B_{L}^{2}\left(q\right)"),
        running_width: sp.Symbol(R"\Gamma(m)"),
    }
)
rel_bw_with_ff_subs

Coupled width [[PDG2020, §Resonances, Eq. (49.21)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=6)]:

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
q0_squared = breakup_momentum_squared(m0 ** 2, m_a, m_b)
ff_sq = BlattWeisskopfSquared(L, z=q_squared * d ** 2)
ff0_sq = BlattWeisskopfSquared(L, z=q0_squared * d ** 2)
rho = phase_space_factor(s, m_a, m_b)
rho0 = phase_space_factor(m0 ** 2, m_a, m_b)
running_width = running_width.subs(
    {
        rho / rho0: sp.Symbol(R"\rho(m)") / sp.Symbol(R"\rho(m_{0})"),
        ff_sq: sp.Symbol("B_{L}^{2}(q)"),
        ff0_sq: sp.Symbol("B_{L}^{2}(q_{0})"),
    },
)
running_width_math = Math(fR"\Gamma(m) = {sp.latex(running_width)}")
running_width_math

with $B_L^2(q)$ the Blatt-Weisskopf form factor as defined [here](https://ampform.readthedocs.io/en/stable/api/ampform.dynamics.html#ampform.dynamics.BlattWeisskopfSquared).

## Phase space factor

We have to choose a phase space factor $\rho$ for the coupled width:

In [ ]:
running_width_math

## Phase space factor ― some options

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
rho = phase_space_factor(s, m_a, m_b)
rho_analytic = phase_space_factor_analytic(s, m_a, m_b)

**Break-up momentum** [[PDG2020, §Kinematics, Eq. (48.1)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-kinematics.pdf#page=3)]:

In [ ]:
q_squared_math = Math(f"q(m) = {sp.latex(sp.sqrt(q_squared, evaluate=False))}")
q_squared_math

'Standard' **phase space factor** [[PDG2020, §Resonances, Eq. (49.8)](https://pdg.lbl.gov/2020/reviews/rpp2020-rev-resonances.pdf#page=4)]:

In [ ]:
rho_subs = rho.subs(sp.sqrt(q_squared), sp.Symbol(R"\sqrt{q^{2}(m)}"))
rho_math = Math(fR"\rho(m) = {sp.latex(rho_subs)}")
rho_math

**Analytic continuation** of the phase space factor [[PDG2016, §Resonances, p.9](https://pdg.lbl.gov/2016/reviews/rpp2016-rev-resonances.pdf#page=9)] (<span style="color:red">only valid if $m_a = m_b$ (?)</span>):

In [ ]:
from ampform.dynamics import _analytic_continuation, phase_space_factor_abs

rho_ac_math = _analytic_continuation(
    sp.Symbol(R"\hat{\rho}"), s, s_threshold=(m_a + m_b) ** 2
)
rho_hat = phase_space_factor_abs(s, m_a, m_b)
rho_hat_subs = rho_hat.subs(
    4 * sp.Abs(q_squared),
    4 * sp.Symbol(R"\left|q^{2}(m)\right|"),
)
rho_ac_math = Math(
    sp.latex(rho_ac_math)
    + R" \quad \mathrm{{with}} \quad \hat{{\rho}}(m) = "
    + sp.latex(rho_hat_subs)
)
rho_ac_math

## Break-up momentum behavior

Notice that the input to the square root is negative for <br>
$|m_a-m_b| < m < m_a+m_b$:

In [ ]:
q_squared_math

In [ ]:
np_breakup_momentum_squared, sliders = symplot.prepare_sliders(
    plot_symbol=m,
    expression=q_squared,
)


def np_breakup_momentum_squared_left(plot_domain, m_a, m_b):
    values = np_breakup_momentum_squared(plot_domain, m_a, m_b)
    return np.where(plot_domain < np.abs(m_a - m_b), values, np.nan)


def np_breakup_momentum_squared_mid(plot_domain, m_a, m_b):
    values = np_breakup_momentum_squared(plot_domain, m_a, m_b)
    return np.where(values < 0, values, np.nan)


def np_breakup_momentum_squared_right(plot_domain, m_a, m_b):
    values = np_breakup_momentum_squared(plot_domain, m_a, m_b)
    return np.where(plot_domain > m_a + m_b, values, np.nan)


plot_domain = np.linspace(0.01, 4, 400)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=1.4,
    m_b=0.9,
)

fig, ax = plt.subplots(figsize=(9, 4.3), tight_layout=True)
fig.canvas.toolbar_visible = False
ax.set_xlabel("$m$")
ax.set_ylabel("$q^2(m)$")
controls = iplt.plot(
    plot_domain,
    np_breakup_momentum_squared_left,
    **sliders,
    ylim=(-1, 3),
    ax=ax,
    label=R"$m \leq m_a-m_b$",
    c="C2",
)
iplt.plot(
    plot_domain,
    np_breakup_momentum_squared_mid,
    controls=controls,
    ylim="auto",
    ax=ax,
    label="$q^2(m) < 0$",
    c="red",
)
iplt.plot(
    plot_domain,
    np_breakup_momentum_squared_right,
    controls=controls,
    ylim="auto",
    ax=ax,
    label=R"$m \geq m_a+m_b$",
    c="C0",
)
ax.set_yticks([0])
plt.axhline(y=0, c="black", linestyle="dotted", linewidth=1)
plt.legend(loc="lower right")
plt.show()

## Analytic continuation

Analytic continuation should take care of those negative values:

In [ ]:
rho_ac_math

In the following slide, we visualize compare this continuation with slightly modified versions of $q$ and $\rho$:

In [ ]:
q_squared_symbol = sp.Symbol("q^{2}(m)")
q_c = ComplexSqrt(q_squared_symbol)
rho_c = phase_space_factor_complex(s, m_a, m_b)

x = sp.Symbol("x", real=True)
complex_sqrt = ComplexSqrt(x)

q_c_symbol = sp.Symbol("q_{c}(m)")
q_c_subs = q_c.subs(4 * q_squared, 4 * q_squared_symbol)
rho_c_subs = rho_c.subs(ComplexSqrt(q_squared), ComplexSqrt(q_squared_symbol))

q_c_latex = fR"{sp.latex(q_c_symbol)} = {sp.latex(q_c)}"
rho_c_latex = fR"\rho_c(m) = {sp.latex(rho_c_subs)}"
display(
    Math(q_c_latex),
    Math(rho_c_latex),
    Math(fR"{sp.latex(complex_sqrt)} = {sp.latex(complex_sqrt.evaluate())}"),
)

## Analytic continuation

In [ ]:
np_rho_c, sliders = symplot.prepare_sliders(plot_symbol=m, expression=rho_c)
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_analytic, "numpy")
np_breakup_momentum = sp.lambdify(
    (m, m_a, m_b), ComplexSqrt(q_squared), "numpy"
)

plot_domain = np.linspace(0, 3, 200)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.6,
    m_b=1.25,
)

fig, axes = plt.subplots(1, 3, figsize=[13, 4], tight_layout=True)
fig.canvas.toolbar_visible = False

ax_q, ax_rho, ax_rho_ac = axes
for ax in axes:
    ax.set_xlabel("$m$")

ylim_q = (-0.04, 1.0)
ylim_rho = (-0.002, 0.05)


def func_imag(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).imag


def func_real(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).real


ax_q.set_title(f"${q_c_latex}$")
controls = iplt.plot(
    plot_domain,
    func_real(np_breakup_momentum),
    label="real",
    **sliders,
    ylim=ylim_q,
    ax=ax_q,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_breakup_momentum),
    label="imaginary",
    controls=controls,
    ylim=ylim_q,
    ax=ax_q,
    alpha=0.7,
)

ax_rho.set_title(f"${rho_c_latex}$")
iplt.plot(
    plot_domain,
    func_real(np_rho_c),
    label="real",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_c),
    label="imaginary",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho,
    alpha=0.7,
)

ax_rho_ac.set_title("analytic")
iplt.plot(
    plot_domain,
    func_real(np_rho_ac),
    label="real",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho_ac,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_ac),
    label="imaginary",
    controls=controls,
    ylim=ylim_rho,
    ax=ax_rho_ac,
    alpha=0.7,
)
plt.legend(loc="upper right")

plt.show()

In [ ]:
rho_ac_math

## Effect on Breit-Wigner

In [ ]:
def breakup_momentum(s: sp.Symbol, m_a: sp.Symbol, m_b: sp.Symbol) -> sp.Expr:
    q_squared = breakup_momentum_squared(s, m_a, m_b)
    return ComplexSqrt(q_squared)


bw_q = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=breakup_momentum,
)
bw_rho = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor_complex,
)
bw_rho_ac = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
    phsp_factor=phase_space_factor_analytic,
)


def make_argand(func):
    def decorated_func(*args, **kwargs):
        values = func(plot_domain, *args, **kwargs)
        argand = np.array([values.real, values.imag])
        return argand.T

    return decorated_func

In [ ]:
np_bw_rho_ac, sliders = symplot.prepare_sliders(
    plot_symbol=m,
    expression=bw_rho_ac.doit(),
)
np_bw_rho = sp.lambdify((m, w0, L, d, m0, m_a, m_b), bw_rho.doit())
np_bw_q = sp.lambdify((m, w0, L, d, m0, m_a, m_b), bw_q.doit())

plot_domain = np.linspace(start=0, stop=4, num=501)
sliders.set_ranges(
    m0=(0, 4, 50),
    Gamma0=(0, 1, 100),
    L=(0, 8),
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
    d=(0, 5),
)


def set_defaults(ev):
    return sliders.set_values(
        m0=1.8,
        Gamma0=0.6,
        L=1,
        m_a=0.7,
        m_b=0.5,
        d=1,
    )


set_defaults(None)
starting_values = widgets.Button(description="defaults")
starting_values.on_click(set_defaults)

zero_angular_momentum = widgets.Button(description="L=0")
zero_angular_momentum.on_click(
    lambda ev: sliders.set_values(
        L=0,
    )
)

below_threshold = widgets.Button(description="below threshold")
below_threshold.on_click(
    lambda ev: sliders.set_values(
        m0=0.95,
        m_a=0.7,
        m_b=0.5,
    )
)

just_below_threshold = widgets.Button(description="just below threshold")
just_below_threshold.on_click(
    lambda ev: sliders.set_values(
        m0=1.04,
        m_a=0.7,
        m_b=0.5,
    )
)

button_box = widgets.HBox(
    [
        starting_values,
        below_threshold,
        just_below_threshold,
        zero_angular_momentum,
    ]
)

fig, all_axes = plt.subplots(
    nrows=3,
    ncols=2,
    figsize=(10, 8),
    gridspec_kw={"width_ratios": [2.5, 1]},
)
fig.suptitle("Relativistic Breit-Wigners with different phase space factors")
fig.canvas.toolbar_visible = False

for ax_left, ax_right in all_axes:
    ax_right.set_ylabel(R"Im$\left(A\right)$")
m_label = "$m$"
ax_left.set_xlabel(m_label)
ax_right.set_xlabel(R"Re$\left(A\right)$")
axes, axes_argand = all_axes.T

axes[0].set_title("Break-up momentum $q$")
axes[1].set_title(R"Phase space factor $\rho$")
axes[2].set_title("analytic continuation")

ylim = "auto"  # (-0.3, 0.8)

controls = iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np_bw_q(*args, **kwargs).real,
    label="real",
    **sliders,
    ylim=ylim,
    ax=axes[0],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho(*args, **kwargs).real,
    label="real",
    controls=controls,
    ylim=ylim,
    ax=axes[1],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho_ac(*args, **kwargs).real,
    label="real",
    controls=controls,
    ylim=ylim,
    ax=axes[2],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_q(*args, **kwargs).imag,
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=axes[0],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho(*args, **kwargs).imag,
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=axes[1],
)
iplt.plot(
    plot_domain.real,
    lambda *args, **kwargs: np_bw_rho_ac(*args, **kwargs).imag,
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=axes[2],
)

for ax in axes:
    iplt.axvline(
        controls["m0"],
        c="red",
        alpha=0.3,
        ax=ax,
        label="$m_0$",
    )
    iplt.axvline(
        lambda m_a, m_b, **kwargs: m_a + m_b,
        controls=controls,
        c="black",
        alpha=0.3,
        ax=ax,
        label="$m_a + m_b$",
    )
    ax.axhline(0, c="black", linewidth=1)
    if isinstance(ylim, tuple):
        ax.set_yticks([0])
axes[0].legend(loc="upper right")

# Argand plots
iplt.scatter(
    make_argand(np_bw_q),
    controls=controls,
    parametric=True,
    c=plot_domain,
    s=1,
    xlim="auto",
    ylim="auto",
    ax=axes_argand[0],
)
iplt.scatter(
    make_argand(np_bw_rho),
    controls=controls,
    parametric=True,
    c=plot_domain,
    s=1,
    xlim="auto",
    ylim="auto",
    ax=axes_argand[1],
)
iplt.scatter(
    make_argand(np_bw_rho_ac),
    controls=controls,
    parametric=True,
    c=plot_domain,
    s=1,
    xlim="auto",
    ylim="auto",
    ax=axes_argand[2],
)
for ax in axes_argand:
    ax.set_xticks([0])
    ax.set_yticks([0])
    plt.colorbar(label=m_label, ax=ax, aspect=30, pad=0.01)
plt.winter()

fig.tight_layout()
plt.show()
display(button_box)

## Formula overview

In [ ]:
display(
    rel_bw_with_ff_subs,
    running_width_math,
    rho_ac_math,
    rho_math,
    q_squared_math,
)